#Colab run example

In [ ]:
! git clone https://github.com/Matan-Lange/Donut.git

In [ ]:
!pip install mlflow
!pip install dicttoxml
!pip install pyngrok
!pip install zss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.6/505.6 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import subprocess
import mlflow
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

In [ ]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Make sure to replace 'YOUR_AUTH_TOKEN' with your actual token
NGROK_AUTH_TOKEN = "insert token here to ngrok to open mlflow ui in clode"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://d575-35-240-184-93.ngrok-free.app


In [ ]:
import sys
from huggingface_hub import HfApi, HfFolder


In [ ]:
import torch
from torch.utils.data import DataLoader, DistributedSampler
import mlflow
from nltk import edit_distance
import numpy as np
import re
import os
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import sys


from huggingface_hub import HfApi, HfFolder
sys.path.append('/content/Donut/')
sys.path.append('/content/Donut/finetune')
from finetune.trainer import TrainerDDP
from finetune.dataset import DonutDatasetFineTune
from finetune.model import load_he_model


os.environ['HF_TOKEN'] = 'insert huggingface token here if repo is private'
repo_name = "jjjlangem/He-Donut"





def setup(rank, world_size):
  os.environ['MASTER_ADDR'] = 'localhost'
  os.environ['MASTER_PORT'] = '12355'
  dist.init_process_group("nccl", rank=rank, world_size=world_size)
  torch.cuda.set_device(rank)

def cleanup():
  dist.destroy_process_group()

def main(rank, world_size):
  setup(rank, world_size)
  processor, model = load_he_model(repo_name = repo_name ,
                                   hf_token = HfFolder.get_token())

  data_path = r'/content/Donut/finetune/data/'
  train_dataset = DonutDatasetFineTune(data_path, max_length=768, processor=processor)

  val_dataset = DonutDatasetFineTune(data_path, split = 'validation', max_length=768, processor=processor)

  train_sampler = DistributedSampler(train_dataset, num_replicas=world_size, rank=rank)
  train_dataloader = DataLoader(train_dataset, batch_size=1, sampler=train_sampler)

  val_sampler = DistributedSampler(val_dataset, num_replicas=world_size, rank=rank)
  val_dataloader = DataLoader(val_dataset, batch_size=1, sampler=val_sampler)

  config = {
      "lr": 3e-5,
      "verbose": True,
      "gradient_clip_val": 1.0,
      'experiment_name': 'fine_tune_donut',
      "mlflow_tracking_uri": 'sqlite:///mlflow.db'  # add tracking uri here

  }

  trainer = TrainerDDP(config, processor, model, train_dataloader, val_dataloader, rank, world_size)

  trainer.train(3)

  cleanup()

  return trainer

if __name__ == "__main__":
  import torch.multiprocessing as mp

  world_size = 1
  if world_size>1 :
      mp.spawn(main, args=(world_size,), nprocs=world_size, join=True)
  else:
      main(0, world_size)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:497: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/5.01k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/processing_utils.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

2024/08/05 19:00:49 INFO mlflow.tracking.fluent: Experiment with name 'fine_tune_donut' does not exist. Creating a new experiment.


train loss: 9.894768714904785 at step 1
train loss: 10.313294410705566 at step 2
train loss: 10.227639198303223 at step 3
train loss: 10.55791187286377 at step 4
train loss: 8.344266891479492 at step 5
train loss: 7.960109233856201 at step 6
train loss: 7.583337783813477 at step 7
train loss: 7.2687788009643555 at step 8
train loss: 7.069705963134766 at step 9
train loss: 6.744565010070801 at step 10
train loss: 6.85011625289917 at step 11
train loss: 6.464442253112793 at step 12
train loss: 6.520479679107666 at step 13
train loss: 6.454214096069336 at step 14
train loss: 6.271694660186768 at step 15
train loss: 6.116786956787109 at step 16
train loss: 6.094271659851074 at step 17
train loss: 6.009284496307373 at step 18
train loss: 5.985904693603516 at step 19
train loss: 5.854194641113281 at step 20
train loss: 5.6938958168029785 at step 21
train loss: 5.709439277648926 at step 22
train loss: 5.566524505615234 at step 23
train loss: 5.431795597076416 at step 24
train loss: 5.37018966

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Prediction: <root><receipcecececececececececececeipceip/20/20/20>רוני שלום אלול אלול 03, דולב, 3supplier>אשר עלי כתוברד צורדוף צורדוף, צורית, צורית, 59upplier></item</item ברשג בשג 2<<<<<</item< ברשגוויז<</item<>>>>>>>>>>>>>>>הוווויזי< הוווני< הווווני</item<<</item</item>>>></item< לולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולולוב</item>>
    Answer: <receipt_number>2666</receipt_number><date>21/04/2020</date><client><name>רוני שלום</name><address>אלול 03, דולב, 3925359</address></client><supplier><name>אשר עלי</name><address>הרדוף 44, צורית, 5917053</address></supplier><items><item><name>ברשג</name><quantity>2</quantity><price>138.87</price><total>277.74</total></item><item><name>קוויז</name><quantity>9</quantity><price>38.68</price><total>348.12</total></item><item><name>התידם</name><quantity>1</quantity><price>23.24</price><total>23.24</total></item></items><total_sum>759.45</total_sum>
 Normed ED: 0.7632058287795

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import xml.etree.ElementTree as ET
import zss
from typing import Union, List
from nltk import edit_distance
from torch.utils.data import DataLoader
from finetune.dataset import DonutDatasetFineTune


class Node(zss.Node):
    def __init__(self, label, text='', children=None):
        self.label = label
        self.text = text.strip() if text else ''
        self.children = children or []

    def get_children(self):
        return self.children

    def get_label(self):
        return self.label

    def get_text(self):
        return self.text

class XMLParseEvaluator:
    """
    Calculate n-TED(Normalized Tree Edit Distance) based accuracy and F1 accuracy score
    """

    @staticmethod
    def xml_to_tree_node(element):
        """
        Helper function to convert an XML element into a tree node
        """
        children = [XMLParseEvaluator.xml_to_tree_node(child) for child in element]
        return Node(element.tag, element.text, children)

    @staticmethod
    def flatten_xml(element, key=""):
        """
        Convert XML into Non-nested Dictionary-like structure
        """
        flatten_data = []

        def _flatten(element, key=""):
            if len(element):
                for child in element:
                    _flatten(child, f"{key}.{child.tag}" if key else child.tag)
            else:
                flatten_data.append((key, element.text.strip() if element.text else ''))

        _flatten(element)
        return flatten_data

    @staticmethod
    def update_cost(node1: Node, node2: Node):
        """
        Update cost for tree edit distance.
        If both are leaf nodes, calculate string edit distance between two labels.
        If one of them is leaf node, cost is length of string in leaf node + 1.
        If neither are leaf nodes, cost is 0 if label1 is same with label2 otherwise 1
        """
        label1 = node1.label
        label2 = node2.label
        label1_leaf = not node1.get_children()
        label2_leaf = not node2.get_children()
        if label1_leaf and label2_leaf:
            return edit_distance(label1, label2)
        elif label1_leaf:
            return 1 + len(label2)
        elif label2_leaf:
            return 1 + len(label1)
        else:
            return int(label1 != label2)

    @staticmethod
    def insert_and_remove_cost(node: Node):
        """
        Insert and remove cost for tree edit distance.
        If leaf node, cost is length of label name.
        Otherwise, 1
        """
        label = node.label
        if not node.get_children():
            return len(label)
        else:
            return 1

    def cal_f1(self, preds: List[str], answers: List[str]):
        """
        Calculate global F1 accuracy score (field-level, micro-averaged) by counting all true positives, false negatives and false positives
        """
        total_tp, total_fn_or_fp = 0, 0
        for pred_xml, answer_xml in zip(preds, answers):
            pred_tree = ET.fromstring(pred_xml)
            answer_tree = ET.fromstring(answer_xml)
            pred_flat = self.flatten_xml(pred_tree)
            answer_flat = self.flatten_xml(answer_tree)
            for field in pred_flat:
                if field in answer_flat:
                    total_tp += 1
                    answer_flat.remove(field)
                else:
                    total_fn_or_fp += 1
            total_fn_or_fp += len(answer_flat)
        return total_tp / (total_tp + total_fn_or_fp / 2)

    def construct_tree_from_xml(self, element: ET.Element, node_name: str = None):
        """
        Convert XML Element into Tree
        """
        if node_name is None:
            node_name = element.tag

        node = Node(node_name, element.text)

        for child in element:
            child_node = self.construct_tree_from_xml(child)
            node.children.append(child_node)

        return node

    def cal_acc(self, pred_xml: str, answer_xml: str):
        """
        Calculate normalized tree edit distance(nTED) based accuracy.
        1) Construct tree from XML,
        2) Get tree distance with insert/remove/update cost,
        3) Divide distance with GT tree size (i.e., nTED),
        4) Calculate nTED based accuracy. (= max(1 - nTED, 0 ).
        """
        pred_element = ET.fromstring(pred_xml)
        answer_element = ET.fromstring(answer_xml)

        pred_tree = self.construct_tree_from_xml(pred_element)
        answer_tree = self.construct_tree_from_xml(answer_element)

        return max(
            0,
            1
            - (
                zss.distance(
                    pred_tree,
                    answer_tree,
                    get_children=zss.Node.get_children,
                    insert_cost=self.insert_and_remove_cost,
                    remove_cost=self.insert_and_remove_cost,
                    update_cost=self.update_cost,
                    return_operations=False,
                )
                / zss.distance(
                    self.construct_tree_from_xml(ET.Element("<root>")),
                    answer_tree,
                    get_children=zss.Node.get_children,
                    insert_cost=self.insert_and_remove_cost,
                    remove_cost=self.insert_and_remove_cost,
                    update_cost=self.update_cost,
                    return_operations=False,
                )
            ),
        )



with torch.no_grad():
  processor, model = load_he_model(repo_name = repo_name ,
                                    hf_token = HfFolder.get_token())

  model.load_state_dict(torch.load('/content/donut_model2.pt'))
  model.eval()
  data_path = r'/content/Donut/finetune/data/'
  test_dataset = DonutDatasetFineTune(data_path, split = 'test', max_length=768, processor=processor)
  test_loader = DataLoader(test_dataset, batch_size=4)
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  model.to(device)

  p = []
  a = []

  for batch in test_loader:
    pixel_values, labels, answers = batch
    pixel_values, labels = pixel_values.to(device), labels.to(device)

    batch_size = pixel_values.shape[0]
    decoder_input_ids = torch.full((batch_size, 1), model.config.decoder_start_token_id,
                                    device=device)

    outputs = model.generate(pixel_values,
                                          decoder_input_ids=decoder_input_ids,
                                          max_length=768,
                                          early_stopping=True,
                                          pad_token_id=processor.tokenizer.pad_token_id,
                                          eos_token_id=processor.tokenizer.eos_token_id,
                                          use_cache=True,
                                          num_beams=1,
                                          bad_words_ids=[[processor.tokenizer.unk_token_id]],
                                          return_dict_in_generate=True)

    predictions = []
    for seq in processor.tokenizer.batch_decode(outputs.sequences):
        seq = seq.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "").replace(processor.tokenizer.bos_token, "")
        seq = re.sub(r"<.*?>", "", seq, count=1).strip()
        predictions.append(seq)

    scores = []
    for pred, answer in zip(predictions, answers):
        answer = answer.replace(processor.tokenizer.eos_token, "")
        score = edit_distance(pred, answer) / max(len(pred), len(answer))
        scores.append(score)


        print(f"Prediction: {pred}")
        print(f"    Answer: {answer}")
        print(f" Normed ED: {score}")

        p.append(pred)
        a.append(answer)



/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:497: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/processing_utils.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:3101: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is se

Prediction: <receipt_number>7173</receipt_number> <date> 26/01/2024 </date> <client> <name> נועה מור </name> <address> הל&quot;ה 73, נירן, 8149364 </address> </client> <supplier> <name> מתן חמוד-גליק </name> <address> הר כנען 41, כפר יונה, 140790 </address> </supplier><items><item> <name> למרקוח </name> <quantity> 8 </quantity> <price> 30.19 </price> <total> 241.52 </total> </item><item> <name> דול </name> <quantity> 6 </quantity> <price> 68.73 </price> <total> 412.38 </total> </item><item> <name> וטעד </name> <quantity> 2 </quantity> <price> 149.53 </price> <total> 299.06 </total> </item></items><total_sum>1114.96</total_sum> </root>
    Answer: <root>
  <receipt_number>7173</receipt_number>
  <date>26/01/2024</date>
  <client>
    <name>נועה מור</name>
    <address>הל&quot;ה 73, נירן, 8149364</address>
  </client>
  <supplier>
    <name>מתן חמוד-גליק</name>
    <address>הר כנען 41, כפר יונה, 1400790</address>
  </supplier>
  <items>
    <item>
      <name>למרקוח</name>
      <quantit

KeyboardInterrupt: 

In [ ]:
p1 = ['<root>' +a for a in p]

evaluator = XMLParseEvaluator()
#accuracy = evaluator.cal_acc(p, a)
f1_score = evaluator.cal_f1(a[a,p)

# print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")

F1 Score: 0.9178217821782179
